# **Analysis Flow: Electric Vehicle Title and Registration Activities Project**

# **Phase 1: Data preparation and exploration**

## **Data import**

In [1]:
SELECT 
Clean_Alternative_Fuel_Vehicle_Type AS 'Electric_Vehicle_Type',DOL_Vehicle_ID,Model_Year,Make,Model,Vehicle_Primary_Use,Electric_Range,Odometer_Reading,
Odometer_Code,New_or_Used_Vehicle AS 'Vehicle_Condition',Sale_Price,Sale_Date,Base_MSRP,Transaction_Type,DOL_Transaction_Date,Transaction_Year,County,City,
State_of_Residence AS 'State',Postal_Code
INTO Electric_Vehicle_Registrations FROM PortfolioProjects.dbo.Electric_Vehicle_Title_and_Registration_Activity

SELECT
County,City,State,Postal_Code,Model_Year,Make,Model,Electric_Vehicle_Type,Clean_Alternative_Fuel_Vehicle_CAFV_Eligibility AS 'Alternative Fuel Eligibility',
Electric_Range,Base_MSRP,DOL_Vehicle_ID
INTO Electric_Vehicle_Population FROM PortfolioProjects.dbo.Electric_Vehicle_Population_Data

(808591 rows affected)

(143596 rows affected)

Total execution time: 00:00:02.543

## **Data cleaning and preprocessing**

### **Detetcing missing values**

In [2]:
SET NOCOUNT ON

DECLARE @TableName NVARCHAR(128) = 'Electric_Vehicle_Registrations';
DECLARE @ColumnName NVARCHAR(128);
DECLARE @SQL NVARCHAR(MAX);

DECLARE ColumnCursor CURSOR FOR
SELECT COLUMN_NAME
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = @TableName;

OPEN ColumnCursor;
FETCH NEXT FROM ColumnCursor INTO @ColumnName;

WHILE @@FETCH_STATUS = 0
BEGIN

	SET @SQL =' SELECT COUNT(*) AS ''The count of null values in '+QUOTENAME(@ColumnName)+'''
	FROM PortfolioProjects.dbo.Electric_Vehicle_Registrations WHERE ' + QUOTENAME(@ColumnName)  + 'IS NULL'
	EXEC sp_executesql @SQL
    
    FETCH NEXT FROM ColumnCursor INTO @ColumnName;
END;

CLOSE ColumnCursor;
DEALLOCATE ColumnCursor;

Commands completed successfully.

Total execution time: 00:00:03.555

The count of null values in [Electric_Vehicle_Type]
0


The count of null values in [DOL_Vehicle_ID]
0


The count of null values in [Model_Year]
0


...

The count of null values in [Transaction_Year]
0


The count of null values in [County]
36


The count of null values in [City]
71


The count of null values in [State]
1


The count of null values in [Postal_Code]
48


### **Checking the column \['Alternative Fuel Eligibility'\]**

In [3]:
SELECT TOP 10 Electric_Range,[Alternative Fuel Eligibility]
FROM PortfolioProjects.dbo.Electric_Vehicle_Population
ORDER BY 1 asc

Commands completed successfully.

Total execution time: 00:00:00.077

Electric_Range,Alternative Fuel Eligibility
0,Eligibility unknown as battery range has not been researched
0,Eligibility unknown as battery range has not been researched
0,Eligibility unknown as battery range has not been researched
0,Eligibility unknown as battery range has not been researched
0,Eligibility unknown as battery range has not been researched
0,Eligibility unknown as battery range has not been researched
0,Eligibility unknown as battery range has not been researched
0,Eligibility unknown as battery range has not been researched
0,Eligibility unknown as battery range has not been researched
0,Eligibility unknown as battery range has not been researched


In [4]:
UPDATE PortfolioProjects.dbo.Electric_Vehicle_Population
SET [Alternative Fuel Eligibility] = 'Unknown'
WHERE [Alternative Fuel Eligibility] = 'Eligibility unknown as battery range has not been researched'

Commands completed successfully.

Total execution time: 00:00:00.635

In [5]:
SELECT DISTINCT TOP 10 TAB2.DOL_Vehicle_ID,TAB2.Electric_Range,TAB1.[Alternative Fuel Eligibility]
FROM PortfolioProjects.dbo.Electric_Vehicle_Population AS TAB1
JOIN PortfolioProjects.dbo.Electric_Vehicle_Registrations AS TAB2
ON TAB1.DOL_Vehicle_ID = TAB2.DOL_Vehicle_ID

Commands completed successfully.

Total execution time: 00:00:00.304

DOL_Vehicle_ID,Electric_Range,Alternative Fuel Eligibility
161704854,0,Unknown
233909490,13,Not eligible due to low battery range
276006153,107,Clean Alternative Fuel Vehicle Eligible
220773423,208,Clean Alternative Fuel Vehicle Eligible
235033431,0,Unknown
240240276,0,Unknown
141313168,0,Unknown
207098538,153,Clean Alternative Fuel Vehicle Eligible
107117524,84,Clean Alternative Fuel Vehicle Eligible
124507946,215,Clean Alternative Fuel Vehicle Eligible


### **Handling a missing column in Electric\_Vehicle\_Registrations table**

In [6]:
ALTER TABLE PortfolioProjects.dbo.Electric_Vehicle_Registrations
ADD [Alternative Fuel Eligibility] VARCHAR(55)

Commands completed successfully.

Total execution time: 00:00:00.010

In [7]:
UPDATE PortfolioProjects.dbo.Electric_Vehicle_Registrations
SET [Alternative Fuel Eligibility] = 
CASE
WHEN Electric_Range = 0  THEN 'Unknown'
WHEN Electric_Range < 30 THEN 'Not eligible due to low battery range'
ELSE 'Clean Alternative Fuel Vehicle Eligible'
END

Commands completed successfully.

Total execution time: 00:00:07.966

## **Basic descriptive statistics**

In [8]:
CREATE PROCEDURE usp_Basic_Descriptive_Statistics
@column_name VARCHAR(55)
AS
BEGIN
DECLARE @sql NVARCHAR(MAX)

SET @sql = 
'
SELECT MIN('+QUOTENAME(@column_name)+') AS ''Min of '+QUOTENAME(@column_name)+''',
AVG('+QUOTENAME(@column_name)+') AS ''Average of '+QUOTENAME(@column_name)+''',
MAX('+QUOTENAME(@column_name)+') AS ''Maximum of '+QUOTENAME(@column_name)+''',
STDEV('+QUOTENAME(@column_name)+') AS ''Standard Deviation of '+QUOTENAME(@column_name)+'''
FROM PortfolioProjects.Dbo.Electric_Vehicle_Registrations
'
EXEC sp_executesql @sql
END

Commands completed successfully.

Total execution time: 00:00:00.008

### 

- **Electric range**

In [9]:
usp_Basic_Descriptive_Statistics @column_name = 'Electric_Range'

Commands completed successfully.

Total execution time: 00:00:00.155

Min of [Electric_Range],Average of [Electric_Range],Maximum of [Electric_Range],Standard Deviation of [Electric_Range]
0,92.95042240143657,337,95.42923121125293


### 

- <span style="font-size: 16.38px;"><b>Odometer reading</b></span>

In [10]:
usp_Basic_Descriptive_Statistics @column_name = 'Odometer_Reading'

Commands completed successfully.

Total execution time: 00:00:00.118

Min of [Odometer_Reading],Average of [Odometer_Reading],Maximum of [Odometer_Reading],Standard Deviation of [Odometer_Reading]
0,2889.7465220364807,962619,11839.12483577425


### 

- <span style="font-size: 16.38px;"><b>Sale price</b></span>

In [11]:
usp_Basic_Descriptive_Statistics @column_name = 'Sale_Price'

Commands completed successfully.

Total execution time: 00:00:00.119

Min of [Sale_Price],Average of [Sale_Price],Maximum of [Sale_Price],Standard Deviation of [Sale_Price]
0,11045.782480883414,12312016,30148.70569646945


### 

- <span style="font-size: 16.38px;"><b>Base manufacturer's suggested retail price</b></span>

In [12]:
usp_Basic_Descriptive_Statistics @column_name = 'Base_MSRP'

Commands completed successfully.

Total execution time: 00:00:00.165

Min of [Base_MSRP],Average of [Base_MSRP],Maximum of [Base_MSRP],Standard Deviation of [Base_MSRP]
0,2883.0776189692933,845000,13764.3006884115


# **Phase 2: Electric vehicle trends**

## **Crafting a Supporting Table**

In [13]:
WITH CTE AS
(
SELECT DOL_Vehicle_ID,Electric_Vehicle_Type,Electric_Range,Make,[Alternative Fuel Eligibility] 
FROM PortfolioProjects.Dbo.Electric_Vehicle_Population

UNION

SELECT DOL_Vehicle_ID,Electric_Vehicle_Type,Electric_Range,Make,[Alternative Fuel Eligibility]
FROM PortfolioProjects.Dbo.Electric_Vehicle_Registrations
)

SELECT *,'#############' AS 'Electric range group'  INTO SupportTable FROM CTE

Commands completed successfully.

Total execution time: 00:00:01.434

### **Detecting the redundant transaction IDs**

In [14]:
SELECT * FROM SupportTable WHERE DOL_Vehicle_ID in
(
SELECT DISTINCT(DOL_Vehicle_ID) FROM SupportTable
GROUP BY DOL_Vehicle_ID
HAVING COUNT(DOL_Vehicle_ID) > 1
)

Commands completed successfully.

Total execution time: 00:00:00.178

DOL_Vehicle_ID,Electric_Vehicle_Type,Electric_Range,Make,Alternative Fuel Eligibility,Electric range group
192417662,Battery Electric Vehicle (BEV),203,PORSCHE,Clean Alternative Fuel Vehicle Eligible,#############
192417662,Battery Electric Vehicle (BEV),0,PORSCHE,Unknown,#############


### **Deleting the redundant transaction ID**

In [15]:
DELETE FROM SupportTable
WHERE DOL_Vehicle_ID = '192417662' AND [Alternative Fuel Eligibility] = 'Unknown'

Commands completed successfully.

Total execution time: 00:00:00.216

## **Electric vehicle adoption over time**

In [16]:
SELECT Transaction_Year,COUNT(*) AS 'Number of transactions' FROM PortfolioProjects.dbo.Electric_Vehicle_Registrations
GROUP BY Transaction_Year ORDER BY 1 ASC

Commands completed successfully.

Total execution time: 00:00:00.114

Transaction_Year,Number of transactions
2010,110
2011,2488
2012,4953
2013,12842
2014,17821
2015,23283
2016,31284
2017,43123
2018,62001
2019,77177


## **Market share analysis**

In [17]:
SELECT Electric_Vehicle_Type,COUNT(DOL_Vehicle_ID) AS 'Count of vehicles' FROM SupportTable
GROUP BY Electric_Vehicle_Type ORDER BY 2 DESC

Commands completed successfully.

Total execution time: 00:00:00.147

Electric_Vehicle_Type,Count of vehicles
Battery Electric Vehicle (BEV),141220
Plug-in Hybrid Electric Vehicle (PHEV),42671
Hydrogen Powered Vehicle,2


In [18]:
SELECT Transaction_Year,Electric_Vehicle_Type,COUNT(*) AS 'Number of transactions' FROM PortfolioProjects.dbo.Electric_Vehicle_Registrations
GROUP BY Transaction_Year,Electric_Vehicle_Type ORDER BY 1 ASC

Commands completed successfully.

Total execution time: 00:00:00.221

Transaction_Year,Electric_Vehicle_Type,Number of transactions
2010,Battery Electric Vehicle (BEV),104
2010,Plug-in Hybrid Electric Vehicle (PHEV),6
2011,Battery Electric Vehicle (BEV),2216
2011,Plug-in Hybrid Electric Vehicle (PHEV),272
2012,Battery Electric Vehicle (BEV),2838
2012,Plug-in Hybrid Electric Vehicle (PHEV),2115
2013,Plug-in Hybrid Electric Vehicle (PHEV),3668
2013,Battery Electric Vehicle (BEV),9174
2014,Plug-in Hybrid Electric Vehicle (PHEV),5505
2014,Battery Electric Vehicle (BEV),12316


## **Most popular electric vehicle models**

In [19]:
SELECT TOP 7 Model,COUNT(*) AS 'Number of transactions' FROM PortfolioProjects.dbo.Electric_Vehicle_Registrations
GROUP BY Model ORDER BY 2 DESC

Commands completed successfully.

Total execution time: 00:00:00.098

Model,Number of transactions
Leaf,146406
Model 3,130404
Model Y,84954
Model S,65003
Volt,52242
Bolt EV,29723
Model X,28680


In [20]:
SELECT Transaction_Year,Model,COUNT(*) AS 'Number of transactions' FROM PortfolioProjects.Dbo.Electric_Vehicle_Registrations
GROUP BY Transaction_Year,Model
HAVING Model in (SELECT TOP 5 Model FROM PortfolioProjects.dbo.Electric_Vehicle_Registrations GROUP BY Model ORDER BY COUNT(*) DESC)
ORDER BY 1 ASC

Commands completed successfully.

Total execution time: 00:00:00.268

Transaction_Year,Model,Number of transactions
2011,Volt,269
2011,Leaf,2086
2012,Volt,1429
2012,Leaf,2525
2012,Model S,78
2013,Volt,2070
2013,Model S,2116
2013,Leaf,6472
2014,Leaf,9125
2014,Model S,2212


# **Phase 3: Electric vehicle characteristics analysis**

## **Electric range analysis**

In [21]:
SELECT TOP 15 DOL_Vehicle_ID,Electric_Range,Electric_Vehicle_Type 
FROM SupportTable WHERE Electric_Range <> 0 ORDER BY 1 DESC

Commands completed successfully.

Total execution time: 00:00:00.059

DOL_Vehicle_ID,Electric_Range,Electric_Vehicle_Type
479254772,38,Plug-in Hybrid Electric Vehicle (PHEV)
479114996,220,Battery Electric Vehicle (BEV)
478935460,75,Battery Electric Vehicle (BEV)
478934571,289,Battery Electric Vehicle (BEV)
478933080,208,Battery Electric Vehicle (BEV)
478926346,25,Plug-in Hybrid Electric Vehicle (PHEV)
478925947,238,Battery Electric Vehicle (BEV)
478925163,220,Battery Electric Vehicle (BEV)
478924358,76,Battery Electric Vehicle (BEV)
478916028,220,Battery Electric Vehicle (BEV)


In [22]:
UPDATE SupportTable
SET [Electric range group] =
CASE

WHEN Electric_Range = 0    THEN 'not-reported'
WHEN Electric_Range <= 50  THEN ' 0-50'
WHEN Electric_Range <= 100 THEN ' 50-100'
WHEN Electric_Range <= 150 THEN '100-150'
WHEN Electric_Range <= 200 THEN '150-200'
WHEN Electric_Range <= 250 THEN '200-250'
WHEN Electric_Range <= 300 THEN '250-300'
WHEN Electric_Range <= 350 THEN '300-350'

END

Commands completed successfully.

Total execution time: 00:00:01.334

In [23]:
SELECT [Electric range group],COUNT(DOL_Vehicle_ID) AS 'Count of vehicles' FROM SupportTable
GROUP BY [Electric range group] ORDER BY 1 ASC

Commands completed successfully.

Total execution time: 00:00:00.214

Electric range group,Count of vehicles
0-50,37206
50-100,23820
100-150,5169
150-200,3751
200-250,28443
250-300,8076
300-350,3218
not-reported,74210


## **Manufacturer analysis**

In [24]:
SELECT Make,COUNT(DOL_Vehicle_ID) AS 'Count of vehicles' FROM SupportTable
GROUP BY Make ORDER BY 2 DESC

Commands completed successfully.

Total execution time: 00:00:00.115

Make,Count of vehicles
TESLA,79982
NISSAN,21626
CHEVROLET,15918
FORD,9990
BMW,8370
KIA,7259
TOYOTA,6284
VOLKSWAGEN,4549
VOLVO,3956
JEEP,3542


## **Vehicle eligibility for alternative fuel status analysis**

In [25]:
SELECT [Alternative Fuel Eligibility],COUNT(DOL_Vehicle_ID) AS 'Count of vehicles' FROM SupportTable
GROUP BY [Alternative Fuel Eligibility] ORDER BY 2 DESC

Commands completed successfully.

Total execution time: 00:00:00.082

Alternative Fuel Eligibility,Count of vehicles
Clean Alternative Fuel Vehicle Eligible,87010
Unknown,74210
Not eligible due to low battery range,22673


# **Phase 4: Electric vehicle transactions analysis**

In [26]:
SELECT DISTINCT(Transaction_Type),COUNT(*) AS 'Number of transactions' FROM PortfolioProjects.Dbo.Electric_Vehicle_Registrations
GROUP BY Transaction_Type ORDER BY 2 DESC

Commands completed successfully.

Total execution time: 00:00:00.083

Transaction_Type,Number of transactions
Registration Renewal,368326
Original Registration,181745
Original Title,179601
Transfer Title,40578
Registration at time of Transfer,38341


# **Phase 5: Geographic analysis**

## **Number of declared cities,Counties,States**

In [27]:
SELECT COUNT(DISTINCT(City)) AS 'N° of cities',COUNT(DISTINCT(County)) AS 'N° of counties',COUNT(DISTINCT(State)) AS 'N° of states'
FROM PortfolioProjects.Dbo.Electric_Vehicle_Registrations
WHERE State NOT IN ('QC','BC','AP','AE','AB','ON','XX')

Warning: Null value is eliminated by an aggregate or other SET operation.

Total execution time: 00:00:00.291

N° of cities,N° of counties,N° of states
1098,324,49


## **Geographical distribution**

In [28]:
SELECT TOP 15 City,County,State,COUNT(*) as 'Number of transactions' FROM PortfolioProjects.dbo.Electric_Vehicle_Registrations
GROUP BY City,County,State
HAVING State NOT IN ('QC','BC','AP','AE','AB','ON','XX')
ORDER BY 4 DESC

Commands completed successfully.

Total execution time: 00:00:00.162

City,County,State,Number of transactions
SEATTLE,King,WA,145339
BELLEVUE,King,WA,44538
REDMOND,King,WA,33085
VANCOUVER,Clark,WA,27612
KIRKLAND,King,WA,25931
SAMMAMISH,King,WA,25237
RENTON,King,WA,20288
BOTHELL,Snohomish,WA,18985
OLYMPIA,Thurston,WA,18105
TACOMA,Pierce,WA,17138


# **Phase 6: Preparing data for visualizations**

## 

- **View 1:**

In [29]:
DROP VIEW IF  EXISTS Geographical_distribution
GO

CREATE VIEW Geographical_distribution AS

SELECT City,County,State,COUNT(*) as 'Number of transactions' FROM PortfolioProjects.dbo.Electric_Vehicle_Registrations
GROUP BY City,County,State
HAVING State NOT IN ('QC','BC','AP','AE','AB','ON','XX')

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.042

## 

- **View 2:**

In [30]:
DROP VIEW IF EXISTS Car_brand_popularity
GO

CREATE VIEW Car_brand_popularity AS

SELECT TOP 8 Make,COUNT(DOL_Vehicle_ID) AS 'Count of vehicles' FROM PortfolioProjects.dbo.SupportTable
GROUP BY Make ORDER BY 2 DESC

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.005

## 

- **View 3:**

In [31]:
DROP VIEW IF EXISTS Top_electric_models
GO

CREATE VIEW Top_electric_models AS

SELECT Transaction_Year,Model,COUNT(*) AS 'Number of transactions' FROM PortfolioProjects.Dbo.Electric_Vehicle_Registrations
GROUP BY Transaction_Year,Model
HAVING Model in (SELECT TOP 6 Model FROM PortfolioProjects.dbo.Electric_Vehicle_Registrations GROUP BY Model ORDER BY COUNT(*) DESC)

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.002

## 

- **View 4:**

In [32]:
DROP VIEW IF EXISTS Electric_vehicle_transactions_over_Time
GO

CREATE VIEW Electric_vehicle_transactions_over_Time AS

SELECT Transaction_Year,Electric_Vehicle_Type,COUNT(*) AS 'Number of transactions' FROM PortfolioProjects.dbo.Electric_Vehicle_Registrations
GROUP BY Transaction_Year,Electric_Vehicle_Type

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.041

- **View 5**

In [33]:
DROP VIEW IF EXISTS Eligibility_status
GO

CREATE VIEW Eligibility_status AS

SELECT [Alternative Fuel Eligibility],COUNT(DOL_Vehicle_ID) AS 'Count of vehicles' FROM SupportTable
GROUP BY [Alternative Fuel Eligibility]

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.002

- **View 6:**

In [34]:
DROP VIEW IF EXISTS Electric_range_group_vehicle_Count
GO

CREATE VIEW Electric_range_group_vehicle_Count AS

SELECT [Electric range group],Electric_Vehicle_Type,COUNT(DOL_Vehicle_ID) AS 'Count of vehicles' FROM SupportTable
GROUP BY [Electric range group],Electric_Vehicle_Type

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.076

## 

- **View 7:**

In [35]:
DROP VIEW IF EXISTS Electric_range_among_categories
GO

CREATE VIEW Electric_range_among_categories AS

SELECT Electric_Range,Electric_Vehicle_Type FROM SupportTable
WHERE Electric_Range <> 0

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.039